# Overview (Example: Twitter)

## Requirement clarification

- Do users post tweets and follow other people?
- Do users search tweets?
- Need to display user timeline?
- Need to push notification on new tweets?
- Need to display trending topics?
- Do tweets contain photos and videos?
- Are we discussing backend only or frontend too?

## System estimation

- Numer of tweets? Number of users? 
- Size of storage?
- Network bandwidth?

## System interface

- What APIs are expected?

## Data model

- User (UserID, Name, Email, CreationDate, LastLogin, etc)
- Tweet (TweetID, Content, TimeStamp, etc)
- UserFollow (UserID1, UserID2)

## High-level design

- 